# Kinds of Kindness: simulations code
This notebook contains the code necessary to create the simulated dataset for a (noisy) utility maximzer characterised by a utility function such as the CES on a given set of constraints.

In [1]:
import sys
sys.path.append('/Users/federicobassi/Desktop/TI.nosync/MPhil_Thesis/python/utils')
from utils.data_generation import *
from utils.data_plotting import *
from utils.test_rev_pref import *
from utils.maximize_utility_bc import *
from utils.budget_placement import *
import seaborn as sns
import matplotlib as mpl
from min_sigma_garp import min_sigma_for_garp
mpl.rcParams.update({"font.family": "serif", "font.serif": ["Times New Roman"],"axes.titlesize": 18, "axes.titleweight": "bold",})
import matplotlib.pyplot as plt
import numpy as np
import pickle
np.random.seed(123)

## Setup

### Paths

In [2]:
path = "/Users/federicobassi/Desktop/TI.nosync/MPhil_Thesis"
img_path = path+"/plots/simulations"
out = path + "/simulated_data"

### Parameters distribution

In [3]:
# Parameters
param_distribution_ces = {'alpha_ces': lambda: np.random.uniform(0.5, 1),
                              'rho_ces': lambda: np.random.uniform(-0.5, 0.95)}

### Budget constraints

In [4]:
andreoni_miller_budgets = [(120, 40), (40, 120), (120, 60), (60, 120), (150, 75),
                               (75, 150), (60, 60), (100, 100), (80, 80), (160, 40), (40, 160)]

andreoni_miller_budgets = [(10*ms, 10*mo) for ms, mo in andreoni_miller_budgets]

with open(out+"/budget_constraints_alg2.pkl", "rb") as f:
    budget_constraints_alg2 = pickle.load(f)

#del budget_constraints_alg2[-1]

upward_sloping = [(10, 130), (20, 110), (40, 70), (70,40), (110,20), (130, 10)]

In [5]:
andreoni_miller_budgets

[(1200, 400),
 (400, 1200),
 (1200, 600),
 (600, 1200),
 (1500, 750),
 (750, 1500),
 (600, 600),
 (1000, 1000),
 (800, 800),
 (1600, 400),
 (400, 1600)]

In [6]:
budget_constraints_alg2

[(400, 2000),
 (420, 1680),
 (470, 1410),
 (500, 1250),
 (510, 1020),
 (560, 800),
 (600, 750),
 (610, 610),
 (630, 420),
 (640, 320),
 (660, 220),
 (680, 170)]

In [7]:
#plot_simple_budget_constraints(andreoni_miller_budgets, title="Andreoni & Miller constraints", save=True, savepath=out, colors="blue")
#plot_simple_upward_constraints(upward_sloping, colors = "black", save="True", savepath=img_path+"/upward.png", title="Upward sloping constraints")
#plot_simple_budget_constraints(budget_constraints_alg2, title="12 new budget constraints", save=True, savepath=out, colors="red")

## Data simulation

In [8]:
df_free_am, df_nodis_am = simulate_dataset(
        budgets = andreoni_miller_budgets,
        utility_func = ces,
        param_distributions=param_distribution_ces,
        n_samples= 200,
        noise_sd = [0,10,20,50,100],
        maximiser="exact",
        seed=42)

In [9]:
"""
df_free_nb_appr_1, df_nodis_nb_appr_1 = simulate_dataset(
        budgets = new_budgets_approach_1,
        utility_func = ces,
        param_distributions=param_distribution_ces,
        n_samples= 200,
        noise_sd = [0,1,2,5,10],
        maximiser="exact")
"""

df_free_nb_appr_2, df_nodis_nb_appr_2 = simulate_dataset(
        budgets = budget_constraints_alg2,
        utility_func = ces,
        param_distributions=param_distribution_ces,
        n_samples= 200,
        noise_sd = [0,10,20,50,100],
        maximiser="exact",
        seed=42)

## Plot noise

In [ ]:
df_nodis_am["utility_func"] = ces 
plot_noise(
    df_nodis_am,
    utility_func=ces,
    noise_sd=50,
    free_disposal=False,
    save=True,
    savepath=img_path,
    title = "Simulation results - CES Utility - α ~ U(0.5,1), ρ ~ U(-0.5,0.95)"
)

In [ ]:
df_free_am["utility_func"] = ces 
plot_noise(
    df_free_am,
    utility_func=ces,
    noise_sd=100,
    free_disposal=False,
    save=True,
    savepath=img_path,
    title = "Simulation results - CES Utility - α ~ U(0.5,1), ρ ~ U(-0.5,0.95)"
)

In [ ]:
"""
df_nodis_nb_appr_2["utility_func"] = ces 
plot_noise(
    df_nodis_nb_appr_2,
    utility_func=ces,
    param_distributions= param_distribution_ces,
    noise_sd=5,
    free_disposal=False,
    save=False
)
"""

## Test revealed preferences

### Plot the distribution of the CCEI score

In [ ]:
plot_index(df_free_am,
           title="CCEI distribution - Andreoni & Miller constraints ", 
           save_path = img_path+"/ccei_am_freedis")

In [ ]:
plot_index(df_nodis_am,
           title="CCEI distribution - Andreoni & Miller constraints ", 
           save_path = img_path+"/ccei_am_nodis")

In [ ]:
plot_index(df_nodis_nb_appr_2, 
           title="CCEI distribution - 11 new budget constraints",
           save_path = img_path+"/ccei_nbs_alg2_nodis")

In [ ]:
plot_index(df_free_nb_appr_2, 
           title="CCEI distribution - 11 new budget constraints",
           save_path = img_path+"/ccei_nbs_alg2_freedis")

### Plot the distribution of the HMI index

In [ ]:
plot_index(df_nodis_am, 
           index_type="HMI", 
           title="HMI distribution - Andreoni & Miller constraints", 
           save_path = img_path+"/hmi_am", 
           note = "CES: α ~ U(0.5,0.95), ρ ~ U(-0.5,0.95). Simulations: 500. Noise generated without free disposal.")

### Plot CCEI distribution by subject's type

In [ ]:
param_dists = {
    # ─────────────────────  ALTRUISTIC  ─────────────────────
    "altruistic_efficiency_maximiser": {
        "alpha_ces": lambda: np.random.uniform(0.5, 0.6),
        "rho_ces":   lambda: np.random.uniform(0.1, 0.9)
    },
    "altruistic_inequity_averse": {
        "alpha_ces": lambda: np.random.uniform(0.5, 0.6),
        "rho_ces":   lambda: np.random.uniform(-0.9, -0.1)
    },

    # ───────────────────────  SELFISH  ──────────────────────
    "selfish_efficiency_maximiser": {
        "alpha_ces": lambda: np.random.uniform(0.6, 1.0),
        "rho_ces":   lambda: np.random.uniform(0.1, 0.9)
    },
    "selfish_inequity_averse": {
        "alpha_ces": lambda: np.random.uniform(0.6, 1.0),
        "rho_ces":   lambda: np.random.uniform(-0.9, -0.1)
    }
}

In [ ]:
for label, pdist in param_dists.items():
    df_fs, df_nd = simulate_dataset(
        budgets            = andreoni_miller_budgets,
        utility_func       = ces,
        param_distributions= pdist,
        n_samples          = 300,
        noise_sd           = [0, 10, 20, 50, 100],
        maximiser          = "exact",
        seed = 42
    )
    plot_index(df_nd, index_type="CCEI", 
               title=label.replace("_", " ").title()+" - Andreoni & Miller constraints",
               save_path = img_path+f"/{label}_am")

In [ ]:
for label, pdist in param_dists.items():
    df_fs, df_nd = simulate_dataset(
        budgets            = budget_constraints_alg2,
        utility_func       = ces,
        param_distributions= pdist,
        n_samples          = 200,
        noise_sd           = [0, 10, 20, 50, 100],
        maximiser          = "exact",
        seed = 42
    )
    plot_index(df_nd, index_type="CCEI", 
               title=label.replace("_", " ").title()+"- 11 new budget constraints",
               save_path = img_path+f"/{label}_nc")

## Distance checks 

In [ ]:
distance_free, distance_no_disposal = simulate_dataset(
        budgets = andreoni_miller_budgets,
        utility_func = ces,
        param_distributions={'alpha_ces': lambda: np.random.uniform(0.7, 1), 'rho_ces': lambda: np.random.uniform(-0.5, 0.5)},
        n_samples= 200,
        noise_sd = [0,1,2,5,10],
        maximiser="exact")

distance_free_2, distance_no_disposal_2 = simulate_dataset(
        budgets = andreoni_miller_budgets,
        utility_func = ces,
        param_distributions={'alpha_ces': lambda: np.random.uniform(0.5, 0.6), 'rho_ces': lambda: np.random.uniform(-0.5, 0.5)},
        n_samples= 200,
        noise_sd = [0,1,2,5,10],
        maximiser="exact")

In [ ]:
distance_free["utility_func"]= ces
plot_noise(
    distance_free,
    utility_func=ces,
    noise_sd=2,
    free_disposal=True,
    save=True,
    savepath=img_path, 
    title = ": α ~ U(0.7, 1), ρ ~ U(-0.5, 0.5)"
)

In [ ]:
distance_free_2["utility_func"]= ces
plot_noise(
    distance_free_2,
    utility_func=ces,
    noise_sd=2,
    free_disposal=False,
    save=True,
    savepath=img_path, 
    title = ": α ~ U(0.5, 0.6), ρ ~ U(-0.5, 0.5)"
)

In [ ]:
df = compute_distance(distance_free)
print("======================================================================")
df = compute_distance(distance_free_2)

## Quantifying noise levels

In [16]:
expected_new_constraints = 333.6
df_loss = attach_money_metric_loss(df_nodis_nb_appr_2, ces)
summary = summarize_welfare_loss_by_noise(df_loss, id_col="id")

summary["percentage"]=summary["mean_welfare_loss"]/expected_new_constraints


display(summary)

,noise,mean_welfare_loss,mean_welfare_loss_rate,percentage
0,0,3.902301e-13,7.788672e-16,1.169754e-15
1,10,1.138584e+01,1.805430e-02,3.413021e-02
2,20,3.812281e+01,6.395928e-02,1.142770e-01
3,50,1.570368e+02,2.648886e-01,4.707339e-01
4,100,3.988218e+02,6.806689e-01,1.195509e+00


In [19]:
expected_am_constraints = 456.8181818

df_loss_am = attach_money_metric_loss(df_nodis_am, ces)
summary_am = summarize_welfare_loss_by_noise(df_loss_am, id_col="id")

summary_am["percentage"]=summary_am["mean_welfare_loss"]/expected_am_constraints


display(summary_am)

,noise,mean_welfare_loss,mean_welfare_loss_rate,percentage
0,0,8.535039e-13,1.033034e-15,1.868367e-15
1,10,7.349457e+00,8.030171e-03,1.608836e-02
2,20,4.186263e+01,4.908187e-02,9.163959e-02
3,50,1.290621e+02,1.425935e-01,2.825240e-01
4,100,3.379479e+02,3.796505e-01,7.397864e-01


### Sensitivity to noise

In [12]:
result = min_sigma_for_garp(
    budgets=andreoni_miller_budgets,
    utility_func=ces,
    param_distributions=param_distribution_ces,
    n_samples=300,
    sigma_start=1,
    sigma_stop=100.0,
    sigma_step=0.1,
    disposal="nodis",
    maximiser="exact",   
    seed=123,
    mc_reps=1 
)

print(result)


{'sigma_min': 28.2, 'violators': 1, 'share_violators': 0.0033333333333333335, 'disposal': 'nodis', 'mc_rep': 0, 'checked_sigmas': [1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0, 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12.0, 12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13.0, 13.1, 13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14.0, 14.1, 14.2, 14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15.0, 15.1, 15.2, 15.3, 15.4, 15.5, 15.6, 15.7, 15.8, 15.9, 16.0, 16.1, 16.2, 16.3, 16.4, 16.5, 16.6, 16.7, 16.8, 16.9,

In [13]:
result = min_sigma_for_garp(
    budgets=budget_constraints_alg2,
    utility_func=ces,
    param_distributions=param_distribution_ces,
    n_samples=300,
    sigma_start=1,
    sigma_stop=100.0,
    sigma_step=0.1,
    disposal="nodis",     
    maximiser="exact",   
    seed=123,
    mc_reps=1           
)

print(result)


{'sigma_min': 7.0, 'violators': 1, 'share_violators': 0.0033333333333333335, 'disposal': 'nodis', 'mc_rep': 0, 'checked_sigmas': [1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0], 'grid_params': (1.0, 100.0, 0.1)}
